In [1]:
import ConnectionConfig as cc
from delta import DeltaTable
from pyspark.sql.functions import col, hour, explode, from_unixtime, unix_timestamp
cc.setupEnvironment()

In [2]:
spark = cc.startLocalCluster("S1S2FactRide")
spark.getActiveSession()

In [3]:
#Extract factRide1 factRide2
fact_ride_s2 = spark.read.format("delta").load("spark-warehouse/factRidesS2")
fact_ride_s1 = spark.read.format("delta").load("spark-warehouse/factRideS1")

In [4]:
#Transform dimWeather and dimDate
fact_ride_s1.createOrReplaceTempView("factRideS1")
fact_ride_s2.createOrReplaceTempView("factRideS2")

In [5]:
fact_rides_S1_S2 = spark.sql("""
SELECT
        frs1.rideSk, frs1.dateSK, frs2.startLockSK, frs2.endLockSK, frs2.customerSK, frs1.weatherSK, 
        frs1.ride_duration, frs2.rideDistance
        FROM factRideS1 frs1
        JOIN factRideS2 frs2 ON frs1.rideSK = frs2.rideSK
""")

fact_rides_S1_S2.show(20)

+------+------+-----------+---------+----------+---------+-------------+--------------------+
|rideSk|dateSK|startLockSK|endLockSK|customerSK|weatherSK|ride_duration|        rideDistance|
+------+------+-----------+---------+----------+---------+-------------+--------------------+
|     2|  2455|       3166|     6046|8589935603|        3|    -94608000|5.575740415708010000|
|     4|  3916|       5628|     4177|8589949890|        4|          304|1.061420171911090000|
|     5|  3916|       3166|     6046|8589935603|        4|          395|5.575740415708010000|
|     8|  3916|       2571|     1862|8589955202|        4|          677|0.177913231674759000|
|    12|  3916|       7055|     4043|8589969414|        4|          269|2.550810388430940000|
|    15|  3916|       5647|     4309|8589942783|        4|         1143|1.019996722533340000|
|    19|  3916|        522|      989|8589970130|        4|          647|1.011845356165030000|
|    23|  3916|       6496|     2191|8589978169|        4|  

In [6]:
# Define path for the Delta table
fact_ride_path = "spark-warehouse/fact_ride_s1_s2"

fact_rides_S1_S2.write.format("delta").mode("overwrite").save(fact_ride_path)